In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
EPOCHS = 50
BATCH_SIZE = 64

data augumentation

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('/content/drive/MyDrive/torch_tutorial/3min_torch/4ch_dnn/.data',
                          train=True,
                          download=True,
                          transform = transforms.Compose([
                                                         transforms.RandomHorizontalFlip(),
                                                         transforms.ToTensor(),
                                                         transforms.Normalize((0.1307,),(0.3081,))
                          ])),
    batch_size = BATCH_SIZE, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('/content/drive/MyDrive/torch_tutorial/3min_torch/4ch_dnn/.data',
                          train=False,
                          download=True,
                          transform = transforms.Compose([
                                                         transforms.ToTensor(),
                                                         transforms.Normalize((0.1307,),(0.3081,))
                          ])),
    batch_size = BATCH_SIZE, shuffle=True
)


Extracting /content/drive/MyDrive/torch_tutorial/3min_torch/4ch_dnn/.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/drive/MyDrive/torch_tutorial/3min_torch/4ch_dnn/.data/FashionMNIST/raw




Extracting /content/drive/MyDrive/torch_tutorial/3min_torch/4ch_dnn/.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/drive/MyDrive/torch_tutorial/3min_torch/4ch_dnn/.data/FashionMNIST/raw




Extracting /content/drive/MyDrive/torch_tutorial/3min_torch/4ch_dnn/.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/drive/MyDrive/torch_tutorial/3min_torch/4ch_dnn/.data/FashionMNIST/raw




Extracting /content/drive/MyDrive/torch_tutorial/3min_torch/4ch_dnn/.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/drive/MyDrive/torch_tutorial/3min_torch/4ch_dnn/.data/FashionMNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [ ]:
class Net(nn.Module):
  def __init__(self, dropout_p=0.2):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 10)

    self.dropout_p = dropout_p

  def forward(self, x):
    x = x.view(-1, 784)
    x = F.relu(self.fc1(x))

    x = F.dropout(x, training=self.training, p = self.dropout_p)
    x = F.relu(self.fc2(x))
    
    x = F.dropout(x, training=self.training, p=self.dropout_p)
    x = self.fc3(x)
    return x

In [ ]:
model = Net(dropout_p=0.2).to(DEVICE)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
def train(model, train_loader, optimizer):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

In [ ]:
def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(DEVICE), target.to(DEVICE)
      output = model(data)
      test_loss += F.cross_entropy(output, target, reduction='sum').item()

      #맞힌 개수 계산
      pred = output.max(1, keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
for epoch in range(1, EPOCHS + 1):
  train(model, train_loader, optimizer)
  test_loss, test_accuracy = evaluate(model, test_loader)

  print('[{}] TEST LOSS: {:.4f}, ACCURACY: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] TEST LOSS: 0.6405, ACCURACY: 77.60%
[2] TEST LOSS: 0.5336, ACCURACY: 80.30%
[3] TEST LOSS: 0.4880, ACCURACY: 82.19%
[4] TEST LOSS: 0.4531, ACCURACY: 83.61%
[5] TEST LOSS: 0.4364, ACCURACY: 84.02%
[6] TEST LOSS: 0.4210, ACCURACY: 84.69%
[7] TEST LOSS: 0.4212, ACCURACY: 84.66%
[8] TEST LOSS: 0.4027, ACCURACY: 85.57%
[9] TEST LOSS: 0.3947, ACCURACY: 85.91%
[10] TEST LOSS: 0.3910, ACCURACY: 85.98%
[11] TEST LOSS: 0.3791, ACCURACY: 86.36%
[12] TEST LOSS: 0.3766, ACCURACY: 86.38%
[13] TEST LOSS: 0.3707, ACCURACY: 86.59%
[14] TEST LOSS: 0.3700, ACCURACY: 86.68%
[15] TEST LOSS: 0.3645, ACCURACY: 86.91%
[16] TEST LOSS: 0.3611, ACCURACY: 86.79%
[17] TEST LOSS: 0.3709, ACCURACY: 86.52%
[18] TEST LOSS: 0.3550, ACCURACY: 87.04%
[19] TEST LOSS: 0.3547, ACCURACY: 87.24%
[20] TEST LOSS: 0.3465, ACCURACY: 87.48%
[21] TEST LOSS: 0.3463, ACCURACY: 87.45%
[22] TEST LOSS: 0.3526, ACCURACY: 87.16%
[23] TEST LOSS: 0.3415, ACCURACY: 87.67%
[24] TEST LOSS: 0.3448, ACCURACY: 87.30%
[25] TEST LOSS: 0.3378, A